In [1]:
import math
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras import layers
from tensorflow.keras.layers import Input, Conv1D, Conv2D, MaxPooling2D, Activation, Reshape, Bidirectional, LSTM, Dense, Lambda, Layer, Dropout
from tensorflow.keras.optimizers import Adam
%run data_loader.ipynb
%matplotlib inline
np.random.seed(1)

In [2]:
X_train, X_test, Y_train, Y_test, Y_train_encoded, Y_test_encoded = data_loader('words', 'words.txt') 
X_train = X_train/255
X_test = X_test/255
X_train = np.reshape(X_train, (-1, 32, 128, 1))
X_test = np.reshape(X_test, (-1, 32, 128, 1))

In [3]:
class CTCLayer(Layer):
    def __init__(self, name=None):
        super().__init__(name=name)
        self.loss_fn = tf.keras.backend.ctc_batch_cost

    def call(self, y_true, y_pred):
        batch_len = tf.cast(tf.shape(y_true)[0], dtype="int64")
        input_length = tf.cast(tf.shape(y_pred)[1], dtype="int64")
        label_length = tf.cast(tf.shape(y_true)[1], dtype="int64")

        input_length = input_length * tf.ones(shape=(batch_len, 1), dtype="int64")
        label_length = label_length * tf.ones(shape=(batch_len, 1), dtype="int64")

        loss = self.loss_fn(y_true, y_pred, input_length, label_length)
        self.add_loss(loss)

        return y_pred

In [4]:
def build_model():

    input_img = Input(shape=(32, 128, 1), name="image", dtype="float32")
    labels =  Input(name="label", shape=(None,), dtype="float32")

    conv1 = Conv2D(16, (5, 5),activation="relu",kernel_initializer="he_normal", padding="same", name="Conv1")(input_img)
    pool1 = MaxPooling2D((2, 2), name="pool1")(conv1)


    conv2 = Conv2D(32, (5, 5), activation="relu", kernel_initializer="he_normal", padding="same", name="Conv2")(pool1)
    pool2 = MaxPooling2D((2, 2), name="pool2")(conv2)
    
    conv3 = Conv2D(64, (3, 3), activation="relu", kernel_initializer="he_normal", padding="same", name="Conv3")(pool2)
    pool3 = MaxPooling2D((2, 1), name="pool3")(conv3)
    
    conv4 = Conv2D(128, (3, 3), activation="relu", kernel_initializer="he_normal", padding="same", name="Conv4")(pool3)
    pool4 = MaxPooling2D((2, 1), name="pool4")(conv4)
    
    conv5 = Conv2D(256, (3, 3), activation="relu", kernel_initializer="he_normal", padding="same", name="Conv5")(pool4)
    pool5 = MaxPooling2D((2, 1), name="pool5")(conv5)
    
    reshape = Reshape(target_shape = (32, 256), name="reshape")(pool5)
    dropout_layer = Dropout(0.2)(reshape)

    blstm1 = Bidirectional( tf.keras.layers.LSTM(256, return_sequences=True, dropout=0.25))(reshape)
    blstm2 = Bidirectional( tf.keras.layers.LSTM(256, return_sequences=True, dropout=0.25))(blstm1)
    
    dense = Dense(80, activation="softmax", name="dense1")(blstm2)

    output = CTCLayer(name="ctc_loss")(labels, dense)

    model =  Model(inputs=[input_img, labels], outputs=output, name="htr_model_1")
    
    opt = Adam()

    model.compile(optimizer=opt)
    return model


In [5]:
model = build_model()
model.summary()

Model: "htr_model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
image (InputLayer)              [(None, 32, 128, 1)] 0                                            
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 32, 128, 16)  416         image[0][0]                      
__________________________________________________________________________________________________
pool1 (MaxPooling2D)            (None, 16, 64, 16)   0           Conv1[0][0]                      
__________________________________________________________________________________________________
Conv2 (Conv2D)                  (None, 16, 64, 32)   12832       pool1[0][0]                      
________________________________________________________________________________________

In [6]:
history = model.fit(x = [X_train, Y_train_encoded], epochs = 10, verbose = 1)

Train on 12800 samples
Epoch 1/10
12800/12800 [==============================] - 350s 27ms/sample - loss: 15.8509
Epoch 2/10
12800/12800 [==============================] - 353s 28ms/sample - loss: 12.6418
Epoch 3/10
12800/12800 [==============================] - 364s 28ms/sample - loss: 10.7983
Epoch 4/10
12800/12800 [==============================] - 358s 28ms/sample - loss: 9.2423
Epoch 5/10
12800/12800 [==============================] - 361s 28ms/sample - loss: 7.6836
Epoch 6/10
12800/12800 [==============================] - 358s 28ms/sample - loss: 6.1214
Epoch 7/10
12800/12800 [==============================] - 356s 28ms/sample - loss: 4.9807
Epoch 8/10
12800/12800 [==============================] - 359s 28ms/sample - loss: 4.1347
Epoch 9/10
   64/12800 [..............................] - ETA: 6:35 - loss: 4.5678

KeyboardInterrupt: 

In [7]:
prediction_model = tf.keras.models.Model(model.get_layer(name="image").input, model.get_layer(name="dense1").output)
prediction_model.summary()
prediction = prediction_model.predict([X_test, Y_test_encoded])
out = tf.keras.backend.get_value(tf.keras.backend.ctc_decode(prediction, input_length=np.ones(prediction.shape[0])*prediction.shape[1], greedy=True)[0][0])
print(out)
i = 0
for x in out:
    print(Y_test[i])
    for p in x:  
        if int(p) != -1:
            print(char_list[int(p)], end = '')
    print('\n')
    i+=1

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
image (InputLayer)           [(None, 32, 128, 1)]      0         
_________________________________________________________________
Conv1 (Conv2D)               (None, 32, 128, 16)       416       
_________________________________________________________________
pool1 (MaxPooling2D)         (None, 16, 64, 16)        0         
_________________________________________________________________
Conv2 (Conv2D)               (None, 16, 64, 32)        12832     
_________________________________________________________________
pool2 (MaxPooling2D)         (None, 8, 32, 32)         0         
_________________________________________________________________
Conv3 (Conv2D)               (None, 8, 32, 64)         18496     
_________________________________________________________________
pool3 (MaxPooling2D)         (None, 4, 32, 64)         0     

Gonamenl's

intention
infention

:
.

"
,

After
aefter

long
Rony

and
and

earnest
ansent

consideration
consideration

,
,

Her
fher

Majesty's
Majerty's

Government
Government

have
hase

come
cae

to
to

the
the

conclusion
condusian

that
that

it
it

would
woould

be
be

right
sight

for
for

Britain
Britain

to
to

make
naake

a
a

formal
formel

application
opplicatiorn

.
.

.
.

for
for

negotiations
negpotiatioun

with
wiith

a
a

view
whieo

to
to

joining
paining

the
the

Community
Commmaninty

.
.

"
"

At
at

this
this

point
paint

,
,

Mr.
Mr.

Paul
Pal

Williams
Nilians

(
(

Cons.
Cans

,
,

Sunderland
dnderlaend

)
?

called
alled

out
anct

"
"

Shame
Heme

.
.

"
"

Mr.
Mr.

Williams
Niliass

is
is

a
a

close
dae

associate
ausinaiate

of
of

Mr.
M.

Fell
Tall

.
,

He
He

claimed
dasned

that
that

,
,

unless
wolers

we
We

were
waere

in
in

the
He

Common
Comwn

Market
nortest

,
,

we
we

should
stouls

not
not

be
be

able
ale

to
to

play
ths

any
as

pa


This
This

figure
Arpe

has
har

been
beun

surpassed
surparrd

only
oaly

on
on

the
the

eve
ore

of
of

the
the

East
East

German
Gern

disturbances
datestonas

on
an

July
dy

17
I.

,
,

1953
195s

.
An

And
corl

officials
othioinr

fear
ler

it
it

may
uay

be
be

too
too

much
muh

for
to

the
the

city's
wnty'r

refugee
efge

camps
conpr

.
.

They
They

will
wll

overflow
qenths

and
anl

private
prinete

houses
hauw

will
will

have
have

to
to

be
he

used
nel

.
.

Prime
Prime

Minister
Mrinister

after
offfer

Prime
Prime

Minister
Minister

speaks
operats

out
ant

in
in

revulsion
restsion

against
apainst

the
He

South
South

African
Aican

Government's
Govermeds

policy
poticg

of
of

apartheid
opartheid

as
as

we
wae

wait
woil

for
for

the
He

curtain
aurtain

to
to

rise
rie

on
an

the
the

Commonwealth
Commonmealtts

Conference
Conference

in
in

London
Lendon

.
.

Will
1It

it
it

end
and

with
with

South
Pouth

Africa's
Aica's

exclusion
exnctusion

from

has
has

now
now

happened
tepereed

.
.

WILL
woe

the
the

new
ner

valuation
rohction

be
be

enough
Coongh

to
to

correct
corret

Germany's
Gornony's

massive
mnessie

trading
Sheding

surplus
Sarplns

and
annd

choke
chake

off
of

speculation
gpecntation

against
aogoit

dollar
dallar

and
and

pound
poond

?
?f

That
That

is
is

doubtful
donolfel

.
.

If
3

however
howevern

,
,

in
"

addition
addition

to
to

her
her

new
new

good-neighbour
get-lghitar

gesture
gesteny

,
o

Germany
anany

takes
taks

a
a

really
reaty

big
lhy

share
share

in
in

giving
gpring

aid
and

to
to

underdeveloped
wnerderdged

nations
retions

,
,

the
the

world
world

outlook
astlaae

will
will

be
be

brighter
hoingfter

.
b

What
hot

gives
gper

rise
rise

to
to

optimism
optinnisnn

is
is

the
the

sign
sign

that
that

Germany
Cermeny

and
and

the
the

other
oher

leading
leding

Western
Wesdern

nations
notiors

are
are

at
at

long
boy

last
bast

moving
horing

towards
towards

a
a


cns

apartheid
pathel

,
,

will
will

be
t

forced
frrt

back
lut

to
i

their
thae

tribal
dlat

reserves
reers

with
wotl

no
nas

prospect
copet

but
lat

a
a

cramped
crnped

and
ant

primitive
mrntes

existence
sailes

.
.

A
"

good
nod

neighbour
apthor

to
to

those
thoe

Africans
Afriaas

who
ahh

will
will

con-
carr

tinue
has

to
to

live
he

as
an

hewers
has

of
A

wood
aot

and
ant

drawers
doses

of
of

water
ats

in
is

the
te

white
all

areas
apa

of
o

South
tell

Africa
Leor

,
,

without
watal

rights
oth

and
aot

without
withat

hope
be

.
.

THE
PHE

same
sesee

sort
not

of
of

good
pot

neighbour
sattion

that
that

he
to

proved
preat

to
t

be
to

to
t

the
H

Jews
tor

fleeing
thny

from
toor

Hitler
tte

in
is

the
t

thirties
thator

.
.

It
1

was
sar

Dr.
tt

Verwoerd
lseal

who
was

led
lat

a
a

protest
soll

against
spoint

admitting
strallay

any
say

of
of

them
taon

because
lecaar

they
thy

would
would

"
"

defile
dtlh

"
"

the
the

national